# Workflow test for `micaflow`

## Inputs in BIDS

- BIDS directory example: `/data/mica3/BIDS_MICs/rawdata`

- subject for test `sub-HC126`

- session `ses-01`

- T1w input test: `/data/mica3/BIDS_MICs/rawdata/sub-HC126/ses-01/anat/sub-HC126_ses-01_T1w.nii.gz`


## Output structure
where type can be `T1w`, `FLAIR`, or `T2w`.
https://bids-specification.readthedocs.io/en/stable/modality-specific-files/magnetic-resonance-imaging-data.html


BIDS specifications. 
`OUTDIR/sub-SUBJECT/ses-SESSION/anat/sub-SUBJECT_ses-SESSION_TYPE.nii.gz`


## Dependencies
1. freesurfer 7.4.0

2. ANTS maybe yes maybe no

3. python. What packages?

## workflow 
1. Bias field correction (ANTs) 

2. Brain mask `mri_synthstrip` Maybe this step can be avoided and we might be able to use the brain segmentation
3. Normalization

4. Brain segmentation `synth_seg` from freesurfer
5. Registration to MNI


### Intensity Non-uniform correction - N4
```bash
N4BiasFieldCorrection  -d 3 -i "$T1n4" -r -o "$T1n4" -v
```

### Brain mask
```bash
mri_synthstrip -i "$T1nativepro" -o "$T1nativepro_brain" -m "$T1nativepro_mask" --no-csf
```

### Normalization `02_proc_flair`

### Segmentation 'synth_seg'

```bash
# Define variables
out=/host/yeatman/local_raid/rcruces/data/tmp

img_fixed=/data_/mica3/BIDS_PNI/derivatives/micapipe_v0.2.0/sub-PNC019/ses-03/anat/sub-PNC019_ses-03_space-nativepro_T1w_nlm.nii.gz

img_moving=/data_/mica3/BIDS_PNI/derivatives/Denoised_T2star/10PNC_nomask/sub-PNC019_ses-03_B1-corrected_T2starmap_nomask_denoised.nii

id=sub-PNC019_ses-03
outname=${out}/${id}_space-nativepro_T2star.nii.gz

threads=15

# Use synthseg to get the labels of the fixed and moving image
mri_synthseg --i ${img_moving} \
             --o ${out}/${id}_moving_synthseg.nii.gz \
             --parc --threads ${threads} --cpu

mri_synthseg --i ${img_fixed} \
             --o ${out}/${id}_fixed_synthseg.nii.gz \
             --parc --threads ${threads} --cpu

```

### Registration to MNI `easy_reg`
```bash
# Calculate the warpfield and affine transformation with easyreg
mri_easyreg --ref ${img_fixed} \
            --flo ${img_moving} \
            --ref_seg ${out}/${id}_fixed_synthseg.nii.gz \
            --flo_seg ${out}/${id}_moving_synthseg.nii.gz \
            --fwd_field ${out}/${id}_from-moving_to-fixed_mode-image_desc-easyreg.nii.gz \
            --threads ${threads} 
```

In [ ]:
%%bash
out=/host/yeatman/local_raid/cerys/data

img_fixed=/host/yeatman/local_raid/cerys/Downloads/mni_icbm152_nlin_sym_09a/mni_icbm152_t1_tal_nlin_sym_09a.nii

T1w=/data/mica3/BIDS_MICs/rawdata/sub-HC126/ses-01/anat/sub-HC126_ses-01_T1w.nii.gz

## Is this on a native space, i.e. T1str_nat?

id=sub-HC126_ses-01
outname=${out}/${id}_space-nativepro_T1w.nii.gz

atlas= mri_synthseg --i ${img_fixed} \
             --o ${out}/${id}_fixed_synthseg.nii.gz \
             --parc --threads ${threads} --cpu

In [ ]:
## 01_proc-structural.sh variables
T1str_nat="${idBIDS}_space-nativepro_T1w"
T1reo="${tmp}/${T1str_nat}_reo.nii.gz"
T1n4="${tmp}/${T1str_nat}_n4.nii.gz"
T1nativepro="${proc_struct}/${T1str_nat}.nii.gz"
T1nativepro_brain="${proc_struct}/${idBIDS}_space-nativepro_T1w_brain.nii.gz"
T1nativepro_mask="${proc_struct}/${idBIDS}_space-nativepro_T1w_brain_mask.nii.gz" 

In [ ]:
%%bash
# Bias field correction (ANTS)
Do_cmd N4BiasFieldCorrection  -d 3 -i "$T1w" -r -o "$T1w" -v

# Rescale intensity [100,0]
Do_cmd ImageMath 3 "$T1nativepro" RescaleImage "$T1w" 0 100

# If no T1native pro exit_status "something is wrong" exit
if [ ! -f "$T1nativepro" ]; then Error "$T1str_nat was not generated"; Do_cmd exit; fi

In [ ]:
%%bash
# Brainmask
Do_cmd mri_synthstrip -i "$T1nativepro" -o "$T1nativepro_brain" -m "$T1nativepro_mask" --no-csf

# If no T1native pro exit_status "something is wrong" exit
if [ ! -f "$T1nativepro_brain" ]; then Error "$T1str_nat masked was not generated"; Do_cmd exit; else ((Nsteps++)); fi

In [ ]:
%%bash

function Image_threshold() {
  img_in=$1
  thrlo=$2
  thrhi=$3
  img_out=$4

  # Create upper and lower thresholded images
  rnd_str=${RANDOM}
  threshlo="${tmp}/${rnd_str}_threshlo.nii.gz"
  threshhi="${tmp}/${rnd_str}_threshhi.nii.gz"

  ThresholdImage 3 "${img_in}" "${threshlo}" threshlo "${thrlo}"
  ThresholdImage 3 "${img_in}" "${threshhi}" threshhi "${thrhi}"

  # Substract the thresholded images to get the unique thresholded img
  ImageMath 3 "${img_out}" - "${threshlo}" "${threshhi}"

  # Cleanup
  rm "${threshlo}" "${threshhi}"

}


# The the mode of the GM, WM and whole brain
function get_mode() {
  # This functions uses mrhistogram to calculate the mode with awk.
  # Using 1000 bins gets the same results as python
  MRI_img=$1
  MRI_mask=$2
  bins=$3
  # Create histogram file
  hist=${tmp}/hist_masked.txt
  mrhistogram -bins "${bins}" -mask "${MRI_mask}" -ignorezero -nthreads "${threads}" "${MRI_img}" "${hist}"
  # get the index with the max frecuency
  max_val=$(awk -F ',' 'NR==3 { m=$3; p=1; for(i=4;i<=NF;i++) { if ($i>m) { m=$i; p=i-2 } } printf "%d ",p }' "${hist}")
  # bash array of each intensity bin
  intensities=($(awk 'NR==2' "${hist}" | tr -s ',' ' '))
  # get the intensity of the maximun frecuency (aka mode)
  mode=${intensities[$((max_val-1))]}
  # remove tmp file
  rm "${hist}"
  # Print the mode
  echo "${mode}" | awk -F"e" 'BEGIN{OFMT="%10.10f"} {print $1 * (10 ^ $2)}'
}

In [ ]:
%%bash

#------------------------------------------------------------------------------#
# 4 | Get the mode for each tissue
mode_gm=$(get_mode "${flair_N4}" "${flair_mask_gm}" 1000)
mode_wm=$(get_mode "${flair_N4}" "${flair_mask_wm}" 1000)
mode_brain=$(get_mode "${flair_N4}" "${flair_mask}" 1000)

Note "mode_gm    :" "${mode_gm}"
Note "mode_wm    :" "${mode_wm}"
Note "mode_brain :" "${mode_brain}"

#------------------------------------------------------------------------------#
# 5 | Normalize intensities by peak of WM (mode).
# This normalization will center the peak of the WM mode intensity at ZERO.
# Mean mode between GM and WM | BG=(GM_mode+WM_mode)/2.0
BG=$(echo "(${mode_gm}+${mode_wm})/2.0" | bc -l)
# mode difference | mode_diff = np.abs(BG - WM_mode)
mode_diff=$(echo "${BG}-${mode_wm}" | bc); mode_diff=$(echo ${mode_diff#-})
# Normalize array | norm_wm = 100.0 * (array - WM_mode)/(mode_diff)
flair_norm="${tmp}/${idBIDS}_flair_norm.nii.gz"
Do_cmd mrcalc "${flair_N4}" "${mode_wm}" -subtract "${mode_diff}" -div 100 -mul "${flair_norm}"

In [ ]:
%%bash
# Normalization
antsRegistrationSyNQuick.sh -d 3 -f "$img_fixed" -m "$T1nativepro_brain" -o "$T1nativepro_brain" -t s -j 1

In [ ]:
%%bash
# Segmentation
threads=15

mri_synthseg --i ${T1nativepro_brain} \
             --o ${out}/${id}_moving_synthseg.nii.gz \
             --parc --threads ${threads} --cpu

mri_synthseg --i ${T1nativepro_brain} \
             --o ${out}/${id}_fixed_synthseg.nii.gz \
             --parc --threads ${threads} --cpu

In [ ]:
%%bash
# Registration to MNI
mri_easyreg --ref ${img_fixed} \
            --flo ${T1nativepro_brain_warped} \
            --ref_seg ${out}/${id}_fixed_synthseg.nii.gz \
            --flo_seg ${out}/${id}_moving_synthseg.nii.gz \
            --fwd_field ${out}/${id}_from-moving_to-fixed_mode-image_desc-easyreg.nii.gz \
            --threads ${threads}